# Mask R-CNN : Train on a Charge Stability Diagram Dataset

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json

import keras
# import pandas as pd
import skimage as sk
import scipy as sp
import tensorflow as tf
import platform

# from data.process_data import *

# Root directory of the project
ROOT_DIR = os.path.abspath(".")
print(ROOT_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

# Configuration


In [ ]:
class qFlowConfig(Config):
    """Configuration for training on the qFlow dataset.
    Derives from the base Config class and overrides values specific
    to the qFlow dataset.
    """
    # Give the configuration a recognizable name
    NAME = "qFlow"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4
    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # background + 3 regimes (0, 1, 2 QD)

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = qFlowConfig()
config.display()

# Dataset

Load in qFlow data.

Extend the Dataset class and add a method to load the shapes dataset, `load_shapes()`, and override the following methods:

* load_image()
* load_mask()
* image_reference()

### Load CSD

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

class ProcessData():
    def __init__(self):
        self.vgg_format = {}

    def extract_csd(self, path_to_data):
        qflow_data = np.load(path_to_data, allow_pickle=True).item()

        voltages = {"P1": qflow_data['V_P1_vec'], "P2": qflow_data['V_P2_vec']}
        N = len(voltages["P1"])
        M = len(voltages["P2"])
        
        assert N == M

        self.width, self.height = N,M
        self.fileID = os.path.basename(path_to_data).replace(".npy","")
        self.filename = self.fileID+".npy"

        # Extract current, state regimes and gradient
        I = np.array([
            data['current'] for data in qflow_data['output']
        ]).reshape((N,N))

        regimes = np.array([
            data['state'] for data in qflow_data['output']
        ]).reshape((N,N))

        def normalize(matrix):
            mean = matrix.mean()
            std = matrix.std()
            return (matrix - mean) / std

        I = normalize(I)

        csd = {"I": np.array(I,dtype=np.float32), "regimes": regimes}

        return csd
    
    def vgg_annotate_csd(self,csd):

        # First create a list of objects in the image
        object_list = []

        regimes = csd["regimes"]

        labelled_regimes = sk.measure.label(
            regimes, background = -1, connectivity = 1
        )

        regions = sk.measure.regionprops(labelled_regimes)

        regions_list = []
        for index in range(1, labelled_regimes.max()):
                region_dict = {}
                vertices = regions[index].coords
                y, x = vertices.T

                regime = regimes[int(np.average(y)), int(np.average(x))]
                region_dict["shape_name"] = "polygon"
                region_dict["all_points_x"] = x.tolist()
                region_dict["all_points_y"] = y.tolist()
                region_dict["class"] = regime

                regions_list.append(region_dict)
        
        object = {}
        object["filename"] = self.filename
        object["size"] = self.width * self.height
        object["regions"] = regions_list

        object_list.append(object)

        # Now create the VGG format
        
        for object in object_list:
             
            filename = object["filename"]
            size = object["size"]
            regions = object["regions"]

            self.vgg_format[self.fileID] = {}
            self.vgg_format[self.fileID]["filename"] = filename
            self.vgg_format[self.fileID]["size"] = size
            self.vgg_format[self.fileID]["regions"] = {}

            index = 0

            for region in regions:
                region_dict = {}
                shape_attributes = {}

                shape_attributes["name"] = region["shape_name"]
                shape_attributes["all_points_x"] = region["all_points_x"]
                shape_attributes["all_points_y"] = region["all_points_y"]

                region_dict["shape_attributes"] = shape_attributes
                region_dict["region_attributes"] = {"label": region["class"]}

                self.vgg_format[self.fileID]["regions"][str(index)] = region_dict

                index += 1

        self.vgg_format[self.fileID]["file_attributes"] = {}

    def dump_json(self,train_val_split, dataset_size, save_folder):
            num_of_train = int(train_val_split * dataset_size)

            keys, values = zip(*self.vgg_format.items())
            vgg_format_train = dict(zip(keys[:num_of_train], values[:num_of_train]))
            vgg_format_val = dict(zip(keys[num_of_train:], values[num_of_train:]))

            with open(save_folder+"/train/annotations_vgg.json", "w") as f:
                json.dump(vgg_format_train, f, cls=NumpyEncoder)
            with open(save_folder+"/val/annotations_vgg.json", "w") as f:
                json.dump(vgg_format_val, f, cls=NumpyEncoder)


ProcessData = ProcessData()
datapath = "/Users/andrijapaurevic/Documents/uWaterloo/research/mainCSG/QuantumDotControl/data/raw/"
save_dir = "/Users/andrijapaurevic/Documents/uWaterloo/research/mainCSG/QuantumDotControl/autotuning/coarse_tuning/models/mask_rcnn"
dataset_size = 50
train_val_split = 0.8
counter = 0

dataset = []
for filename in os.listdir(datapath):
    if filename.endswith(".npy"):
        if counter == dataset_size: 
            break

        csd = ProcessData.extract_csd(os.path.join(datapath,filename))
        ProcessData.vgg_annotate_csd(csd)

        dataset.append(csd)

        counter += 1

ProcessData.dump_json(train_val_split, dataset_size, save_dir)

# plt.imshow(dataset[0]["I"])
# plt.title("Sample CSD")
# plt.show()

# plt.imshow(dataset[0]["regimes"])
# plt.title("Sample CSD")
# plt.show()




### Create Custom Dataset Class

In [ ]:
class CustomDataset(utils.Dataset):
    def __init__(self):
        super(CustomDataset, self).__init__()

    def load_custom(self, annotations_dir: str, dataset_dir: str, suffix: str):
        assert suffix in ["train", "val"]

        self.add_class("custom", 0, "0 QD")
        self.add_class("custom", 1, "1 QD")
        self.add_class("custom", 2, "2 QD")

        annotations = json.load(
            open(
                os.path.join(annotations_dir, f"{suffix}/annotations_vgg.json")
            )
        )
        annotations = list(annotations.values()) # VIA 2.0 expects list not dict
        annotations = [a for a in annotations if a["regions"]] # Skip unannotated images
        
        for a in annotations:
            polygons = [region['shape_attributes'] for region in a['regions'].values()]
            custom = [region['region_attributes'] for region in a['regions'].values()]

            num_ids = []
            for n in custom:
                try:
                    if n['label'] == 0:
                        num_ids.append(1)
                    elif n['label'] == 1:
                        num_ids.append(2)
                    elif n['label'] == 2:
                        num_ids.append(3)
                except:
                    pass
                        
            image_path = os.path.join(dataset_dir, a['filename'])

            height, width = int(np.sqrt(a['size'])), int(np.sqrt(a['size']))

            self.add_image(
                "custom",
                image_id=a['filename'],
                path = image_path,
                width = width,
                height = height,
                polygons = polygons,
                num_ids = num_ids
            )

    def load_mask(self, image_id):

        image_info = self.image_info[image_id]
        if image_info["source"] != "custom":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = image_info['num_ids']

        mask = np.zeros(
            [image_info["height"], image_info['width'], len(image_info['polygons'])], dtype=np.uint8
        )    

        for i, p in enumerate(image_info["polygons"]):
            rr, cc = sk.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr,cc,i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)

        return mask, num_ids


    def image_reference(self, image_id):
        info = self.image_info[image_id]
        if info["source"] == "custom":
            return info["path"]
        return super(self.__class__,self).image_reference(image_id)
dataset_train = CustomDataset()
dataset_train.load_custom(ROOT_DIR,datapath, "train")
dataset_train.prepare()

dataset_val = CustomDataset()
dataset_val.load_custom(ROOT_DIR,datapath,"val")
dataset_val.prepare()

# Create Model

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

# Training

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2, 
            layers="all")

# Detection

In [ ]:
class InferenceConfig(qFlowConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_val.class_names, figsize=(8, 8))

# Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)

    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)

visualize.display_differences(
image,
gt_bbox, gt_class_id, gt_mask,
r['rois'], r['class_ids'], r['scores'], r['masks'],
dataset_val.class_names,
show_box=False, show_mask=False,
iou_threshold=0.5, score_threshold=0.5)
    
print("mAP: ", np.mean(APs))